In [1]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import RetinaNet

In [2]:
class_num = 80

In [3]:
model = RetinaNet(9, class_num, [3, 4, 6, 3], freeze_bn=False)
my_state_dict = model.state_dict()

In [4]:
my_total_k = []
my_total_v = []
my_convs = []
my_bns = []
for i, (k, v) in enumerate(my_state_dict.items()):
    my_total_k.append(k)
    my_total_v.append(v)
    if 'conv' in k:
        my_convs.append(v)
    if 'bn' in k and "num_batches_tracked" not in k:
        my_bns.append(v)
    # print(f'{i}: {k}\t\t{v.shape}')

In [5]:
len(my_convs), len(my_bns)

(65, 196)

In [6]:
org_state_dict = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/coco_resnet_50_map_0_335_state_dict.pt', map_location='cpu')

In [7]:
# v5_state_dict = org_state_dict['model_state_dict']

In [8]:
org_total_k, org_total_v = [], []
convs = []
bns = []
for i, (k, v) in enumerate(org_state_dict.items()):
    if 'conv' in k:
        convs.append(v)
    if 'bn' in k:
        bns.append(v)
    org_total_k.append(k)
    org_total_v.append(v)
    # print(f'{i}: {k}\t\t{v.shape}')

In [9]:
len(convs), len(bns)

(65, 196)

In [10]:
l = []
for i, j in zip(my_convs, convs):
    l.append(i.shape == j.shape)
print(f"sum(l) = {len(my_convs) == sum(l)}")

sum(l) = True


In [11]:
l = []
for i, j in zip(my_bns, bns):
    l.append(i.shape == j.shape)
print(f"sum(l) = {len(my_bns) == sum(l)}")

sum(l) = True


## load prarmeters

In [12]:
i = 0
for k, v in my_state_dict.items():
    if 'num_batches_tracked' in k:
        continue
    if v.shape != org_total_v[i].shape:
        print(f'i = {i}, {k}, {v.shape}, {org_total_v[i].shape}')
    i += 1

i = 289, classification.output.weight, torch.Size([720, 256, 3, 3]), torch.Size([36, 256, 3, 3])
i = 290, classification.output.bias, torch.Size([720]), torch.Size([36])
i = 299, regression.output.weight, torch.Size([36, 256, 3, 3]), torch.Size([720, 256, 3, 3])
i = 300, regression.output.bias, torch.Size([36]), torch.Size([720])


In [13]:
i = 0
for k, v in my_state_dict.items():
    if 'num_batches_tracked' in k:
        continue
    if v.shape != org_total_v[i].shape:
        print(f'i = {i}, {k}, {v.shape}, {org_total_v[i].shape}')
    i += 1

i = 289, classification.output.weight, torch.Size([720, 256, 3, 3]), torch.Size([36, 256, 3, 3])
i = 290, classification.output.bias, torch.Size([720]), torch.Size([36])
i = 299, regression.output.weight, torch.Size([36, 256, 3, 3]), torch.Size([720, 256, 3, 3])
i = 300, regression.output.bias, torch.Size([36]), torch.Size([720])


In [14]:
i, j = 0, 0
total = org_total_v[:]
for m in model.modules():
    if i in [289, 292, 299, 300]:
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert m.weight.data.shape == total[i].shape, f"{m.weight.data.shape},\t{total[i].shape}"
        m.weight.data = total[i]
        # print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            assert m.bias.data.shape == total[i].shape
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        assert m.weight.data.shape == total[i].shape
        m.weight.data = total[i]
        i += 1
        assert m.bias.data.shape == total[i].shape
        m.bias.data = total[i]
        i += 1
        assert m.running_mean.shape == total[i].shape
        m.running_mean = total[i]
        i += 1
        assert m.running_var.shape == total[i].shape
        m.running_var = total[i]
        i += 1
        # m.num_batches_tracked = total[i]
        # i += 1

In [15]:
i

289

## check

In [16]:
i = 0
total = org_total_v[:]
for m in model.modules():
    if i in [289, 292, 299, 300]:
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        # assert (m.num_batches_tracked == total[i]).all()
        # i += 1

In [17]:
my_output_state = model.cpu().state_dict()
torch.save({'model_state_dict': my_output_state}, "/home/uih/JYL/Programs/YOLO_ckpts/retinanet_for_coco.pth")

In [18]:
st = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/retinanet_for_coco.pth')
model.load_state_dict(st["model_state_dict"])

<All keys matched successfully>